In [1]:
import gzip        
import io            
import numpy as np
import pandas as pd   
import pymysql.cursors 
import sqlite3        
import urllib.request 

import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [2]:
db_params = open("mikepnyu_database_params.txt").read().split()

conn = pymysql.connect(host=db_params[0],
                             user=db_params[1],
                             password=db_params[2],
                             db=db_params[3],
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [3]:
sql3 = """
SELECT Person_Name, Person_Position, Letter_ID, Date_Min, Date_Max, Recipient_Name
FROM (
SELECT Mention.Person_ID AS "Person_ID", CONCAT(Mention.Person_Name, " / ", Mention.PLRE_ID, " / ", Mention.BLZG_ID) AS "Person_Name",
CONCAT(careers.Position, " of ", careers.Location) AS "Person_Position", letters.Letter_ID, letters.Date_Min, letters.Date_Max,
CONCAT(Recipient.Person_Name, " / ", Recipient.PLRE_ID, " / ", Recipient.BLZG_ID) AS "Recipient_Name"
FROM mentions
    LEFT JOIN people AS Mention
        ON mentions.Person_ID = Mention.Person_ID
    LEFT JOIN letters
        ON letters.Letter_ID = mentions.Letter_ID
    LEFT JOIN people AS Recipient
        ON letters.Recipient_ID = Recipient.Person_ID
    LEFT JOIN careers
        ON mentions.Person_ID = careers.Person_ID AND letters.Date_Min >= careers.Date_Min
        AND letters.Date_Max <= careers.Date_Max
UNION ALL
SELECT Recipient.Person_ID AS "Person_ID", CONCAT(Recipient.Person_Name, " / ", Recipient.PLRE_ID, " / ", Recipient.BLZG_ID) AS "Person_Name",
CONCAT(careers.Position, " of ", careers.Location) AS "Person_Position", letters.Letter_ID, letters.Date_Min, letters.Date_Max, 
CONCAT(Recipient.Person_Name, " / ", Recipient.PLRE_ID, " / ", Recipient.BLZG_ID) AS "Recipient_Name"
FROM letters
    LEFT JOIN people AS Recipient
        ON letters.Recipient_ID = Recipient.Person_ID
    LEFT JOIN careers
        ON Recipient.Person_ID = careers.Person_ID AND letters.Date_Min >= careers.Date_Min
        AND letters.Date_Max <= careers.Date_Max
        ) AS query
WHERE Person_ID = "P0005"
ORDER BY Date_Min
"""

df = pd.read_sql(sql3, conn)

df

,Person_Name,Person_Position,Letter_ID,Date_Min,Date_Max,Recipient_Name
0,Acacius / Acacius 8 / Acacius i,Governor of Galatia,F0657,361,361,Acacius / Acacius 8 / Acacius i
1,Acacius / Acacius 8 / Acacius i,Governor of Galatia,F0617,361,361,Modestus / Domitius Modestus 2 /
2,Acacius / Acacius 8 / Acacius i,Governor of Galatia,F0308,361,361,Modestus / Domitius Modestus 2 /
3,Acacius / Acacius 8 / Acacius i,Governor of Galatia,F0622,361,361,Acacius / Acacius 8 / Acacius i
4,Acacius / Acacius 8 / Acacius i,Governor of Galatia,F0651,361,361,Acacius / Acacius 8 / Acacius i
5,Acacius / Acacius 8 / Acacius i,Governor of Galatia,F0658,361,361,Acacius / Acacius 8 / Acacius i
6,Acacius / Acacius 8 / Acacius i,Governor of Galatia,F0621,361,361,Themistius / Themistius 2 /
7,Acacius / Acacius 8 / Acacius i,Governor of Galatia,F0298,361,361,Acacius / Acacius 8 / Acacius i
8,Acacius / Acacius 8 / Acacius i,Governor of Galatia,F0752,362,362,Acacius / Acacius 8 / Acacius i
9,Acacius / Acacius 8 / Acacius i,Governor of Galatia,F0732,362,362,Acacius / Acacius 8 / Acacius i


In [8]:
sql2 = """
SELECT destinations.Destination_ID AS "Destination_ID"
FROM letters, destinations
WHERE letters.Destination_ID = destinations.Destination_ID
ORDER BY destinations.Destination_ID"""

with conn.cursor() as cursor:

    cursor.execute(sql2)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

destinations = pd.DataFrame(result, columns = names)

destinations['Total'] = destinations.groupby(['Destination_ID'])['Destination_ID'].transform('count')

grouped = destinations.drop_duplicates()

grouped = grouped.sort_values(['Total'], ascending=[0])

grouped = grouped.reset_index(drop=True)

grouped

,Destination_ID,Total
0,Constantinople,22
1,Phoenicia,16
2,Galatia,12
3,Sirmium,8
4,Cappadocia,5
5,Euphratensis,5
6,Milan,4
7,Cilicia,4
8,Ancyra,4
9,Egypt,3


In [11]:
import folium

sql = """
SELECT letters.Letter_ID, destinations.Destination_ID, destinations.Destination_Type, destinations.Latitude, 
destinations.Longitude
FROM letters, destinations
WHERE letters.Destination_ID = destinations.Destination_ID
ORDER BY letters.Letter_ID"""

# this is a pretty efficient code block for going from cursor to dataframe
with conn.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

destinations = pd.DataFrame(result, columns = names)

destinations['Total'] = destinations.groupby(['Destination_ID'])['Destination_ID'].transform('count')

libanius_letters = folium.Map(location=[35, 22], zoom_start=4,tiles='http://{s}.tile.openstreetmap.se/hydda/base/{z}/{x}/{y}.png',
                   attr='Tiles courtesy of <a href="http://openstreetmap.se/" target="_blank">OpenStreetMap Sweden</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>')

galatia_geo = '.\Maps\map_Galatia.geojson'
egypt_geo = '.\Maps\map_Egypt2.geojson'
cappadocia_geo = '.\Maps\map_Cappadocia.geojson'
bithynia_geo = '.\Maps\map_Bithynia3.geojson'
armenia_geo = '.\Maps\map_Armenia.geojson'
cilicia_geo = '.\Maps\map_Cilicia.geojson'
euphratensis_geo = '.\Maps\map_Euphratensis.geojson'
palestine_geo = '.\Maps\map_Palestine.geojson'
phoenicia_geo = '.\Maps\map_Phoenicia2.geojson'
arabia_geo = '.\Maps\map_Arabia.geojson'
lycia_geo = '.\Maps\Lycia.geojson'

libanius_letters.choropleth(geo_path=galatia_geo, fill_color="#FF0000", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=egypt_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=cappadocia_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=bithynia_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=armenia_geo, fill_color="#0000FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=cilicia_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=euphratensis_geo, fill_color="#FF0000", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=palestine_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=phoenicia_geo, fill_color="#FF0000", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=arabia_geo, fill_color="#CC00FF", fill_opacity=0.8)
libanius_letters.choropleth(geo_path=lycia_geo, fill_color="#0000FF", fill_opacity=0.8)


for index, row in destinations.iterrows():
    if row["Destination_Type"] == "City":
        if row["Total"] == 1:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", 1 Letter : F" + str(row["Letter_ID"]), 
                      icon = folium.Icon(icon = 'pushpin', color ='blue')).add_to(libanius_letters)
        elif row["Total"] >= 2 and row["Total"] < 5:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'pushpin', color ='purple')).add_to(libanius_letters)
        else:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'pushpin', color ='red')).add_to(libanius_letters)
    if row["Destination_Type"] == "Region":        
        if row["Total"] > 0 and row["Total"] < 2:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", 1 Letter : F" + str(row["Letter_ID"]), 
                      icon = folium.Icon(icon = 'question-sign', color ='blue')).add_to(libanius_letters)
        elif row["Total"] >= 2 and row["Total"] < 5:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'question-sign', color ='purple')).add_to(libanius_letters)
        else:
            folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Destination_ID"]+ ", " + str(row["Total"]) + " Letters", 
                      icon = folium.Icon(icon = 'question-sign', color ='red')).add_to(libanius_letters)
            
folium.Marker([36.20138, 36.161749], popup="Antioch, Home of Libanius and Origin of the Letters",icon = folium.Icon(icon = 'fullscreen', color ='green')).add_to(libanius_letters)

libanius_letters